# Initialization & setup

In [0]:
#google colab
!pip uninstall tensorflow; pip install tensorflow==2.0.0-beta1

In [13]:
#google colab
import sys
ROOT = '/content/drive/My Drive/'
PROJ = 'Tesi/'
LOG = PROJ + 'logs/file.log'
PROJ_COMPLETE_PATH = ROOT + PROJ
LOG_COMPELTE_PATH = ROOT + LOG

if(PROJ_COMPLETE_PATH not in sys.path):
  sys.path.append(PROJ_COMPLETE_PATH)

if(ROOT not in sys.path):
  sys.path.append(ROOT)

sys.path.append(ROOT + 'fuzzylearn/')

sys.path

['/tensorflow-1.15.0/python3.6',
 '',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/My Drive/Tesi/',
 '/content/drive/My Drive/',
 '/content/drive/My Drive/fuzzylearn/',
 '/content/drive/My Drive/fuzzylearn/']

In [0]:
#google colab

### uncomment these line if there isn't already fuzzylearn repo in google drive
#!rm -rf /content/drive/My\ Drive/Tesi
#!mkdir /content/drive/My\ Drive/Tesi/
#!git clone https://tomgeek27@bitbucket.org/tomgeek27/riconoscimento-volti-tesi.git /content/drive/My\ Drive/Tesi/

### uncomment this line if you need to update your repo (updated from develop branch)
#!cd /content/drive/My\ Drive/Tesi; git pull origin develop

# Logging

In [0]:
import logging

def log_init(filename="/content/drive/My Drive/Tesi/logs/file2.log"):

  logger = logging.getLogger(__name__)
  logger.setLevel(logging.INFO)
  if(not logger.hasHandlers()):
    f_handler = logging.FileHandler(filename)
    f_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

    c_handler = logging.StreamHandler()
    c_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
    logger.addHandler(f_handler)
    logger.addHandler(c_handler)

  return logger

def log_info(logger, label, message, *args):
  if(logger != None):
    if(label == None):
      logger.info(message, *args)
    else:
      logger.info("(" + str(label) + ") - " + message, *args)

# Log parser

Here you can find all tools that you need to retrieve information from _file.log_

In [0]:
def str2fuzzifier(f_str):
  if(f_str == 'LinearFuzzifier'):
    return fl.fuzzifiers.LinearFuzzifier
  elif(f_str == 'QuantileLinearPiecewiseFuzzifier'):
    return fl.fuzzifiers.QuantileLinearPiecewiseFuzzifier

def str2kernel(k_str):
  if(k_str == 'GaussianKernel'):
    return fl.kernel.GaussianKernel

def lastAnalyzedClass(lines):
  """Return the label of the last trained class, you could use it for train all model after the last trained"""
  return list(bestParams(lines).keys())[-1]

def bestParams(lines):
  """Return params produced best results"""
  return _searchForValue(lines, "BEST_PARAMS_")

def bestScores(lines):
  """Return best score for all model"""
  return _searchForValue(lines, "BEST_SCORE_")

def params(lines):
  """Return all params used for train all model"""
  return _searchForValue(lines, "Started GridSearchCV")

def params_nested(lines):
  return _searchForValue(lines, "Started Started Nested-GridSearchCV")

def getNestedResults(lines):
  return _searchForValue(lines, "NESTED CROSS VALIDATION with")

def getNestedMean(lines):
  return _searchForValue(lines, "NESTED CROSS VALIDATION (mean)")

def dataDimension(lines):
  """Return the number of feature used for training model"""
  dataDimension_ = _searchForValue(lines, "DATA_DIMENSION_")
  if(dataDimension_ == {}):
    defaultMap = {}

    for i in np.unique(olivetti_labels):
      defaultMap[i] = 2
    
    return defaultMap

  return dataDimension_

def _searchForValue(lines, v):
  m = {}
  for x in lines:
    if(v in x):
      c = x[x.index('(') + 1: x.index(')')]
      m[c] = x[x.index(': ') + 2:-2]
  
  return m

def logLines(log):
  f = open(log, "r")
  return [x for x in f]

def get_params(lines, label):
  line = bestParams(lines)[label]

  line_params = line.split(',')
  c_ = int(line_params[0][line_params[0].index(': ') + 2:])
  fuzzifier_ = line_params[1].split('.')[-1][:-2]

  k_ = line_params[2][6:line_params[2].index('(')]
  param_k_ = float(line_params[2][line_params[2].index('(')+1:line_params[2].index(')')])
  return (c_, fuzzifier_, (k_, param_k_))

# Data processing

In [0]:
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

olivetti = fetch_olivetti_faces()

olivetti_data = olivetti.data
olivetti_labels = olivetti.target

In [0]:
def get_classes(dataset_labels):
  return np.unique(dataset_labels)

def get_tsne_data(dataset, dimension=2):
  return TSNE(n_components=dimension).fit_transform(dataset) if dimension < 4 else TSNE(n_components=dimension).fit_transform(dataset)

def get_pca_data(dataset, dimension=2):
  pca_xd = PCA(n_components=dimension)
  data_xd = pca_xd.fit_transform(dataset) 
  return data_xd

In [19]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedShuffleSplit
import fuzzylearn as fl
import logging
from sklearn.model_selection import GroupKFold
import importlib

importlib.reload(fl)
def estimate_membership_holdout(x, y, c=1, k=fl.kernel.GaussianKernel(sigma=.5), fuzzifier=fl.fuzzifiers.LinearFuzzifier, y_needs_mapping=False, label=""):
  if(y_needs_mapping):
    y = map_labels(y, label)
  model = fl.FuzzyInductor(c=c, k=k, fuzzifier=fuzzifier).fit(x, y)

  return model

def grid_search(x, y, params, data_dimension, data_reduction="t", logger=None, y_needs_mapping=False, label="", folds=3):
  """data_reduction: use "t" for refer to t-SNE, use "p" to refer to PCA"""
  check_label(label)
  
  if(y_needs_mapping):
    y = map_labels(y, label)

  log_info(logger, label, "Started GridSearchCV with: %s", params)
  log_info(logger, label, "DATA_REDUCTION_: %s", data_reduction)
  log_info(logger, label, "DATA_DIMENSION_: %s", data_dimension)
  clf = GridSearchCV(fl.FuzzyInductor(), params, cv=stratified_folds(x, y, folds=folds)).fit(x, y)

  log_info(logger, label, "CV_RESULTS_: %s", clf.cv_results_)
  log_info(logger, label, "BEST_SCORE_: %s", clf.best_score_)
  log_info(logger, label, "BEST_PARAMS_: %s", clf.best_params_)
  return clf


def nested_grid_search(x, y, params, data_dimension, data_reduction="t", logger=None, y_needs_mapping=False, label="", folds=3, innerFolds=3):
  check_label(label)

  log_info(logger, label, "Started Nested-GridSearchCV with: %s", params)
  clf = grid_search(x, y, params, data_dimension, data_reduction=data_reduction, logger=logger, y_needs_mapping=y_needs_mapping, label=label, folds=innerFolds)

  if(y_needs_mapping):
    y = map_labels(y, label)

  results = cross_val_score(clf.best_estimator_, x, y, cv=stratified_folds(x, y, folds=folds), verbose=10)

  log_info(logger, label, "NESTED CROSS VALIDATION with %s fold: %s", folds, results)
  log_info(logger, label, "NESTED CROSS VALIDATION (mean): %s", np.mean(results))

def map_labels(y, label):
    assert(label != "")
    return [1 if elem == label else 0 for elem in y]

def check_label(label):
  assert(label != "" or label is not None) #is important to know which class is training 

def stratified_folds(X, y, random_state=0, folds=3):
  return StratifiedShuffleSplit(n_splits=folds, random_state=random_state).split(X, y)


gurobi not available


# Plot utils

In [0]:
def show_scatter_plot(data, labels):
  for lab in range(40):
          plt.scatter(data[labels==lab, 0],
                      data[labels==lab, 1],
                      label=lab,
                      c=[np.random.rand(3,)],
                      marker="$" + str(lab) + "$",
                      s=200
                      )
  plt.show()

def gr_membership_contour(estimated_membership, data, labels, label, with_label=False, precision=.25):
  plt.figure(figsize=(15, 15))
  data_filtered = [list(elem) for elem, l in list(zip(data, labels)) if l == label]

  offsetX = (maxX(data_filtered) - minX(data_filtered))/2
  offsetY = (maxY(data_filtered) - minY(data_filtered))/2

  x = np.arange(minX(data_filtered) - offsetX, maxX(data_filtered) + offsetX, precision)
  y = np.arange(minY(data_filtered) - offsetY, maxY(data_filtered) + offsetY, precision)
  X, Y = np.meshgrid(x, y)
  zs = np.array([estimated_membership([[x, y]])
                  for x,y in zip(np.ravel(X), np.ravel(Y))])
  Z = zs.reshape(X.shape)

  membership_contour = plt.contour(X, Y, Z,
                                    levels=(.1, .3, .5, .7, .8, .9, .95), colors='k')
  
  if(with_label):
    plt.clabel(membership_contour, inline=1)

  plt.title("Membership contour on test {}".format(label))
  show_scatter_plot(data, labels)

  plt.show()


def minX(x): return min(x, key=lambda t: t[0])[0]

def maxX(x): return max(x, key=lambda t: t[0])[0]

def minY(x): return min(x, key=lambda t: t[1])[1]

def maxY(x): return max(x, key=lambda t: t[1])[1]

# Estimations

In [0]:
def build_best_model(data, labels, filename, c):
  c_, fuzzifier_, (kernel_, param_) = get_params(logLines(filename), str(c))
  return fl.FuzzyInductor(c=c_, fuzzifier=str2fuzzifier(fuzzifier_), k=str2kernel(kernel_)(param_)).fit(data, map_labels(labels, c))

def get_models_on_best_params_holdout(data, labels, filename):
  models = {}
  for c in get_classes(labels):
    c_, fuzzifier_, (kernel_, param_) = get_params(logLines(filename), str(c))
    models[c] = fl.FuzzyInductor(c=c_, fuzzifier=str2fuzzifier(fuzzifier_), k=str2kernel(kernel_)(param_)).fit(data, map_labels(labels, c))

  return models

In [0]:
def get_correct_predictions(train_data, test_data, train_labels, test_labels, models, filename, logger=None):
  predictions_test = []
  predictions_train = []

  for model in models:
    predictions_test.append(models[model].predict(test_data))
    predictions_train.append(models[model].predict(train_data))

  zipped_predictions_test = list(zip(*predictions_test))
  zipped_predictions_train = list(zip(*predictions_train))
  results_test = list(zip([sorted(enumerate(predict), reverse=True, key=lambda x: x[1])[0][0] for predict in zipped_predictions_test], test_labels))
  results_train = list(zip([sorted(enumerate(predict), reverse=True, key=lambda x: x[1])[0][0] for predict in zipped_predictions_train], train_labels))

  res_test = sum([1 if elem[0] == elem[1] else 0 for elem in results_test])/len(test_labels)
  res_train = sum([1 if elem[0] == elem[1] else 0 for elem in results_train])/len(train_labels)
  diff_test = [elem for elem in results_test if elem[0] != elem[1]]
  diff_train = [elem for elem in results_train if elem[0] != elem[1]]
  
  log_info(logger, None, "CORRECT_TEST_PREDICTIONS_: %s", res_test)
  log_info(logger, None, "CORRECT_TRAIN_PREDICTIONS_: %s", res_train)
  log_info(logger, None, "DIFFERENT_TEST_RESULTS_: %s", diff_test)
  log_info(logger, None, "DIFFERENT_TRAIN_RESULTS_: %s", diff_train)

  return (res_test, res_train, diff_test, diff_train)

In [0]:
logger = log_init(filename=ROOT + PROJ + "/logs/of_results_predictions_tsne_2.log")

res_test, res_train, diff_test, diff_train = get_correct_predictions(get_tsne_data(olivetti_data), olivetti_labels, ROOT + PROJ + '/logs/of_tsne_2.log', logger=logger)
res_test, res_train, diff_test, diff_train

# Model selection

---

Here you can find all of the training done with ``` grid_search ``` or ``` nested_grid_search ```



In [0]:
def start_grid_search(data, labels, params, dimension, logger, filename):
  lastAnalyzedClass_ = int(lastAnalyzedClass(logLines(filename)))

  for c in get_classes(labels):
    if(c > lastAnalyzedClass_):
      grid_search(data, labels, params, dimension, data_reduction="p", logger=logger, y_needs_mapping=True, label=c)

def start_nested_grid_search(data, labels, params, dimension, logger, filename):
  lastAnalyzedClass_ = int(lastAnalyzedClass(logLines(filename)))

  for c in get_classes(labels):
    if(c > lastAnalyzedClass_):
      nested_grid_search(data, labels, params, dimension, data_reduction="p", logger=logger, y_needs_mapping=True, label=c)

In [0]:
params = {
    'c': [1, 1000],
    'k': [fl.kernel.GaussianKernel(sigma=.5), fl.kernel.GaussianKernel(sigma=.2)],
    'fuzzifier': [fl.fuzzifiers.LinearFuzzifier, fl.fuzzifiers.QuantileLinearPiecewiseFuzzifier]
}

dimension = 2

logger = log_init()

start(params, dimension, logger)